In [ ]:
import pandas as pd
import autograd.numpy as np
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint, Bounds, NonlinearConstraint
from autograd import grad, jacobian, hessian
from scipy import optimize
import matplotlib.pyplot as plt

# Read Data

In [ ]:
def get_data():
    data = pd.read_csv(f"data/kowacz_data.csv")
    data = pd.concat([data,pd.read_csv(f"data/Kowacz 2007 data.csv")],axis=1)
    Ba = data['Ba'].to_numpy(dtype="float64") * 1e-06
    SO4 = data['SO4'].to_numpy(dtype="float64") * 1e-06
    step_v = data['growth rate'].to_numpy(dtype="float64")
    return Ba, SO4, step_v
    
def get_constants():
    df = pd.read_excel(io='data/Flux_analysis_cleancopy-cz.xlsx',sheet_name='model d')
    hbd_yo = df.iloc[15,34]
    ekt = df.iloc[8,34]
    Ksp = df.iloc[13,34]
    hbd = np.prod(df.iloc[9:12,34].to_numpy(dtype='float64'))
    h = df.iloc[9,34]
    yo = 120 * 10e-09
    hbd_yo = hbd / yo
    constants = { "ekt": ekt, "Ksp": Ksp, "hbd": hbd, "h": h, "yo":yo }
    return constants

# Step Velocity Functions

In [ ]:
def get_v_b(v_a, k_a, k_b, Ksp):
    return Ksp * k_a * k_b / v_a

def find_var(v_a, k_a, k_b, A, B, constants):
    Ksp = constants["Ksp"]
    ekt = constants["ekt"]

    v_b = get_v_b(v_a,k_a,k_b,Ksp)

    S = np.sqrt(A * B / Ksp)

    i_A = 2 * np.exp(-2 * ekt) * (S**2 - 1) * (v_b * k_a * A)/(k_a * A + v_b)
    i_B = 2 * np.exp(-2 * ekt) * (S**2 - 1) * (v_a * k_b * B) / (k_b * B + v_a)
    i_super = (i_A + i_B) / 2
    i_under = ((v_a * v_b * np.exp(-2 * ekt))/((2 - S**2) * v_b + k_a * A) + ((v_a * v_b * np.exp(-2 * ekt)) / ((2 - S**2) * v_a + k_b * B))) * (1 - S**2)

    i = np.maximum(i_super,i_under)

    u = np.abs(2 *(k_a * k_b * A * B - v_a * v_b) / (k_a * A + k_b * B + v_a + v_b))

    rho = np.sqrt(2 * i / u)

    P_a = (k_a * A + v_b) / (k_a * A + k_b * B + v_a + v_b)
    P_b = (k_b * B + v_a) / (k_a * A + k_b * B + v_a + v_b) 

    return rho, u

def find_step_v(v_a, k_a, k_b, A, B, constants):
    h = constants["h"]

    # Change to b
    rho, u = find_var(v_a, k_a, k_b, A, B, constants)
    return rho * u * h * 10**9

def residual_step(x, A, B, step_v, constants):
    return find_step_v(x[0], x[1], x[2], A, B, constants) - step_v

def minimize_step(x, A, B, step_v, constants):
    return np.sum(np.square(residual_step(x, A, B, step_v, constants)))

# Plot Function

In [ ]:
def plot_step_v(step_v_eqn, barite, sulfate, parameters, step_v,constants):
    x = barite / sulfate
    x_sort = x.argsort()

    plt.plot(x, step_v,'o',label="Step Velocity")
    plt.plot(x[x_sort], step_v_eqn(*parameters, barite[x_sort], sulfate[x_sort],constants),'o',label="fit")
    plt.ylabel('Step V')
    plt.xlabel('Barite / Sulfate ratio')
    plt.xscale('log')
    plt.legend()
    plt.show()

In [ ]:
# Get Data
A, B, step_v = util.get_data()
constants = util.get_constants()

# Define Constraints
linear_con = LinearConstraint([0,1,-1], -np.inf, 0)
bounds = Bounds(0,np.inf)

# Initial Value
x0 = np.array([65.08,4.5e05,7.455e06])

#%% Optimization
min_lsq = minimize(minimize_step, x0, jac=jacobian(minimize_step),hess=hessian(minimize_step),args=(A,B,step_v,constants),method='Newton-CG', options={'maxiter':1000})
print(min_lsq)

print("Parameters: v_a={:.3e}, v_b={:.3e}, k_a={:.3e}, k_b={:.3e}".format(min_lsq.x[0], get_v_b(*min_lsq.x, constants["Ksp"]), min_lsq.x[1], min_lsq.x[2]))

#%% Plot Values
plot.plot_step_v(eqn.find_step_v, A, B, min_lsq.x, step_v, constants)

#%% Get Residual value
print("New RSS Value: ",eqn.minimize_step(min_lsq.x, A, B, step_v, constants))
print("Initial RSS Value: ",eqn.minimize_step(x0, A, B, step_v, constants))